In [1]:
import sys
import os
from midi2audio import FluidSynth
import pretty_midi
from os import listdir
import re
from google.cloud import storage
import pickle as pkl
import IPython.display as ipd

In [2]:
sys.path.append('../progs')
from utility_functions import *

# Specify SoundFont for MIDI to WAV Conversion - Currently Default MuseScore

In [3]:
auth_json = '../gcp_bucket_auth/fyr-bucket-auth.json'
bucket_name = 'fyr-audio-data'
#gcp_filepath = 'sound_fonts/default_sound_font.sf2'
SoundFont = '/home/webban_berkeley_edu/default_sound_font.sf2'

# create storage client
storage_client = storage.Client.from_service_account_json(auth_json)
# get bucket with name
bucket = storage_client.get_bucket(bucket_name)
# get bucket data as blob
#blob = bucket.get_blob(gcp_filepath)
# download as temporary midi file
#blob.download_to_filename('tempFile.sf2')

# create a FluidSynth instance with the temporary sound font
#fs = FluidSynth('tempFile.sf2')
#fs9000 = FluidSynth('tempFile.sf2', sample_rate=9000)

fs = FluidSynth(SoundFont)
fs9000 = FluidSynth(SoundFont, sample_rate=9000)

#os.remove('tempFile.sf2')

## Midi Dictionaries
Source: https://www.midi.org/specifications/item/gm-level-1-sound-set

In [4]:
midi_classes = {35: 'Acoustic Bass Drum', 36: 'Bass Drum 1', 37: 'Side Stick', 38: 'Acoustic Snare', 39: 'Hand Clap', 
                40: 'Electric Snare', 41: 'Low Floor Tom', 42: 'Closed Hi Hat', 43: 'High Floor Tom', 44: 'Pedal Hi-Hat', 
                45: 'Low Tom', 46: 'Open Hi-Hat', 47: 'Low-Mid Tom', 48: 'Hi-Mid Tom', 49: 'Crash Cymbal 1', 
                50: 'High Tom',  51: 'Ride Cymbal 1', 52: 'Chinese Cymbal', 53: 'Ride Bell', 54: 'Tambourine', 
                55: 'Splash Cymbal', 56: 'Cowbell', 57: 'Crash Cymbal 2', 58: 'Vibraslap', 59: 'Ride Cymbal 2', 
                60: 'Hi Bongo', 61: 'Low Bongo', 62: 'Mute Hi Conga', 63: 'Open Hi Conga', 64: 'Low Conga', 
                65: 'High Timbale', 66: 'Low Timbale', 67: 'High Agogo', 68: 'Low Agogo', 69: 'Cabasa', 
                70: 'Maracas', 71: 'Short Whistle', 72: 'Long Whistle', 73: 'Short Guiro', 74: 'Long Guiro', 
                75: 'Claves', 76: 'Hi Wood Block', 77: 'Low Wood Block', 78: 'Mute Cuica', 79: 'Open Cuica', 
                80: 'Mute Triangle', 81: 'Open Triangle',
               # to be removed
                22: 'Unknown 22', 26: 'Unknown 26'
               }

# Generate labels for eGMD data

In [5]:
# get a list of raw egmd midi files
storage_client = storage.Client.from_service_account_json(auth_json)
egmd_midi_files = [blob.name for blob in storage_client.list_blobs(bucket,prefix='raw-audio/eGMD/') 
                   if blob.name[-5:]=='.midi']

In [6]:
len(egmd_midi_files)

45547

In [11]:
start_i = 45546
end_i = 40000
i = start_i

## Warning - the below cell takes a LONG (~60 hours) time to process all ~45k midi files (less if you don't do sr 9000 degraded files)

In [7]:
dict_pkl_path = 'raw-audio/reverseOrderPklFiles/eGMDdict2.pkl'
data_pkl_path = 'raw-audio/reverseOrderPklFiles/eGMDlabels2.pkl'

In [8]:
dictEGMD = readGcpPkl(auth_json, bucket_name, dict_pkl_path)
labels = readGcpPkl(auth_json, bucket_name, data_pkl_path)

In [9]:
min(list(dictEGMD.keys()))

35002

In [10]:
start_i = 35001
end_i = 33000
i = start_i

In [11]:
dictEGMD = {}
labels = []
#i = 0 # song index number

for file in egmd_midi_files[start_i:end_i:-1]:
    drummer = file.split('/')[-3]
    subdir = file.split('/')[-2]
    song_name = file.split('/')[-1]
    
    # midi2labels imported from utility_functions
    l = midi2labels(auth_json=auth_json, bucket=bucket_name, gcp_filepath=file, alt=True)
    fname = drummer + '_' + subdir + '_' + song_name
    
    # update eGMD dict and labels list
    dictEGMD[i] = fname
    l = [i, l]
    labels.append(l)
    i -= 1
    
    # convert the midi file to a wav file
    # read in the midi file from GCP as blob
    blob = bucket.get_blob(file)
    blob.download_to_filename('tempFile.midi')
    # use FluidSynth with SondFont file to convert midi to wav
    fs.midi_to_audio('tempFile.midi', 'tempFile.wav')
    # small sample rate for purposefully degraded audio
    fs9000.midi_to_audio('tempFile.midi','tempFileDegraded.wav')
    # use gsutil to write both wav files to GCP bucket
    !gsutil -m cp tempFile.wav gs://fyr-audio-data/raw-audio/eGMD/eGMD-wavfiles/{fname + '.wav'}
    !gsutil -m cp tempFileDegraded.wav gs://fyr-audio-data/raw-audio/eGMD/eGMD-wavfiles/sr9000/{fname + '.wav'}
    os.remove('tempFile.midi')
    os.remove('tempFile.wav')
    os.remove('tempFileDegraded.wav')

#ipd.Audio('tempFile.wav')

Copying file://tempFile.wav [Content-Type=audio/x-wav]...
/ [1/1 files][ 18.2 MiB/ 18.2 MiB] 100% Done                                    
Operation completed over 1 objects/18.2 MiB.                                     
Copying file://tempFileDegraded.wav [Content-Type=audio/x-wav]...
/ [1/1 files][  3.7 MiB/  3.7 MiB] 100% Done                                    
Operation completed over 1 objects/3.7 MiB.                                      
Copying file://tempFile.wav [Content-Type=audio/x-wav]...
/ [1/1 files][ 18.2 MiB/ 18.2 MiB] 100% Done                                    
Operation completed over 1 objects/18.2 MiB.                                     
Copying file://tempFileDegraded.wav [Content-Type=audio/x-wav]...
/ [1/1 files][  3.7 MiB/  3.7 MiB] 100% Done                                    
Operation completed over 1 objects/3.7 MiB.                                      
Copying file://tempFile.wav [Content-Type=audio/x-wav]...
/ [1/1 files][ 18.2 MiB/ 18.2 MiB] 100% Do

Copying file://tempFileDegraded.wav [Content-Type=audio/x-wav]...
/ [1/1 files][234.0 KiB/234.0 KiB] 100% Done                                    
Operation completed over 1 objects/234.0 KiB.                                    
Copying file://tempFile.wav [Content-Type=audio/x-wav]...
/ [1/1 files][  1.1 MiB/  1.1 MiB] 100% Done                                    
Operation completed over 1 objects/1.1 MiB.                                      
Copying file://tempFileDegraded.wav [Content-Type=audio/x-wav]...
/ [1/1 files][234.0 KiB/234.0 KiB] 100% Done                                    
Operation completed over 1 objects/234.0 KiB.                                    
Copying file://tempFile.wav [Content-Type=audio/x-wav]...
/ [1/1 files][  1.1 MiB/  1.1 MiB] 100% Done                                    
Operation completed over 1 objects/1.1 MiB.                                      
Copying file://tempFileDegraded.wav [Content-Type=audio/x-wav]...
/ [1/1 files][234.0 KiB/234.0 KiB]

Copying file://tempFile.wav [Content-Type=audio/x-wav]...
/ [1/1 files][  1.1 MiB/  1.1 MiB] 100% Done                                    
Operation completed over 1 objects/1.1 MiB.                                      
Copying file://tempFileDegraded.wav [Content-Type=audio/x-wav]...
/ [1/1 files][234.0 KiB/234.0 KiB] 100% Done                                    
Operation completed over 1 objects/234.0 KiB.                                    
Copying file://tempFile.wav [Content-Type=audio/x-wav]...
/ [1/1 files][  1.1 MiB/  1.1 MiB] 100% Done                                    
Operation completed over 1 objects/1.1 MiB.                                      
Copying file://tempFileDegraded.wav [Content-Type=audio/x-wav]...
/ [1/1 files][234.0 KiB/234.0 KiB] 100% Done                                    
Operation completed over 1 objects/234.0 KiB.                                    
Copying file://tempFile.wav [Content-Type=audio/x-wav]...
/ [1/1 files][  1.1 MiB/  1.1 MiB] 100% Do

Copying file://tempFileDegraded.wav [Content-Type=audio/x-wav]...
/ [1/1 files][114.5 KiB/114.5 KiB] 100% Done                                    
Operation completed over 1 objects/114.5 KiB.                                    
Copying file://tempFile.wav [Content-Type=audio/x-wav]...
/ [1/1 files][558.8 KiB/558.8 KiB] 100% Done                                    
Operation completed over 1 objects/558.8 KiB.                                    
Copying file://tempFileDegraded.wav [Content-Type=audio/x-wav]...
/ [1/1 files][114.5 KiB/114.5 KiB] 100% Done                                    
Operation completed over 1 objects/114.5 KiB.                                    
Copying file://tempFile.wav [Content-Type=audio/x-wav]...
/ [1/1 files][558.8 KiB/558.8 KiB] 100% Done                                    
Operation completed over 1 objects/558.8 KiB.                                    
Copying file://tempFileDegraded.wav [Content-Type=audio/x-wav]...
/ [1/1 files][114.5 KiB/114.5 KiB]

Copying file://tempFile.wav [Content-Type=audio/x-wav]...
/ [1/1 files][  7.4 MiB/  7.4 MiB] 100% Done                                    
Operation completed over 1 objects/7.4 MiB.                                      
Copying file://tempFileDegraded.wav [Content-Type=audio/x-wav]...
/ [1/1 files][  1.5 MiB/  1.5 MiB] 100% Done                                    
Operation completed over 1 objects/1.5 MiB.                                      
Copying file://tempFile.wav [Content-Type=audio/x-wav]...
/ [1/1 files][  7.4 MiB/  7.4 MiB] 100% Done                                    
Operation completed over 1 objects/7.4 MiB.                                      
Copying file://tempFileDegraded.wav [Content-Type=audio/x-wav]...
/ [1/1 files][  1.5 MiB/  1.5 MiB] 100% Done                                    
Operation completed over 1 objects/1.5 MiB.                                      
Copying file://tempFile.wav [Content-Type=audio/x-wav]...
/ [1/1 files][  7.4 MiB/  7.4 MiB] 100% Do

...

Copying file://tempFileDegraded.wav [Content-Type=audio/x-wav]...
/ [1/1 files][101.3 KiB/101.3 KiB] 100% Done                                    
Operation completed over 1 objects/101.3 KiB.                                    
Copying file://tempFile.wav [Content-Type=audio/x-wav]...
/ [1/1 files][494.3 KiB/494.3 KiB] 100% Done                                    
Operation completed over 1 objects/494.3 KiB.                                    
Copying file://tempFileDegraded.wav [Content-Type=audio/x-wav]...
/ [1/1 files][101.3 KiB/101.3 KiB] 100% Done                                    
Operation completed over 1 objects/101.3 KiB.                                    
Copying file://tempFile.wav [Content-Type=audio/x-wav]...
/ [1/1 files][494.3 KiB/494.3 KiB] 100% Done                                    
Operation completed over 1 objects/494.3 KiB.                                    
Copying file://tempFileDegraded.wav [Content-Type=audio/x-wav]...
/ [1/1 files][101.3 KiB/101.3 KiB]

Copying file://tempFile.wav [Content-Type=audio/x-wav]...
/ [1/1 files][532.8 KiB/532.8 KiB] 100% Done                                    
Operation completed over 1 objects/532.8 KiB.                                    
Copying file://tempFileDegraded.wav [Content-Type=audio/x-wav]...
/ [1/1 files][109.3 KiB/109.3 KiB] 100% Done                                    
Operation completed over 1 objects/109.3 KiB.                                    
Copying file://tempFile.wav [Content-Type=audio/x-wav]...
/ [1/1 files][532.8 KiB/532.8 KiB] 100% Done                                    
Operation completed over 1 objects/532.8 KiB.                                    
Copying file://tempFileDegraded.wav [Content-Type=audio/x-wav]...
/ [1/1 files][109.3 KiB/109.3 KiB] 100% Done                                    
Operation completed over 1 objects/109.3 KiB.                                    
Copying file://tempFile.wav [Content-Type=audio/x-wav]...
/ [1/1 files][532.8 KiB/532.8 KiB] 100% Do

Copying file://tempFileDegraded.wav [Content-Type=audio/x-wav]...
/ [1/1 files][336.3 KiB/336.3 KiB] 100% Done                                    
Operation completed over 1 objects/336.3 KiB.                                    
Copying file://tempFile.wav [Content-Type=audio/x-wav]...
/ [1/1 files][  1.6 MiB/  1.6 MiB] 100% Done                                    
Operation completed over 1 objects/1.6 MiB.                                      
Copying file://tempFileDegraded.wav [Content-Type=audio/x-wav]...
/ [1/1 files][336.3 KiB/336.3 KiB] 100% Done                                    
Operation completed over 1 objects/336.3 KiB.                                    
Copying file://tempFile.wav [Content-Type=audio/x-wav]...
/ [1/1 files][  1.6 MiB/  1.6 MiB] 100% Done                                    
Operation completed over 1 objects/1.6 MiB.                                      
Copying file://tempFileDegraded.wav [Content-Type=audio/x-wav]...
/ [1/1 files][336.3 KiB/336.3 KiB]

Copying file://tempFile.wav [Content-Type=audio/x-wav]...
/ [1/1 files][  1.7 MiB/  1.7 MiB] 100% Done                                    
Operation completed over 1 objects/1.7 MiB.                                      
Copying file://tempFileDegraded.wav [Content-Type=audio/x-wav]...
/ [1/1 files][351.5 KiB/351.5 KiB] 100% Done                                    
Operation completed over 1 objects/351.5 KiB.                                    
Copying file://tempFile.wav [Content-Type=audio/x-wav]...
/ [1/1 files][  1.7 MiB/  1.7 MiB] 100% Done                                    
Operation completed over 1 objects/1.7 MiB.                                      
Copying file://tempFileDegraded.wav [Content-Type=audio/x-wav]...
/ [1/1 files][351.5 KiB/351.5 KiB] 100% Done                                    
Operation completed over 1 objects/351.5 KiB.                                    
Copying file://tempFile.wav [Content-Type=audio/x-wav]...
/ [1/1 files][  1.7 MiB/  1.7 MiB] 100% Do

Copying file://tempFileDegraded.wav [Content-Type=audio/x-wav]...
/ [1/1 files][  1.1 MiB/  1.1 MiB] 100% Done                                    
Operation completed over 1 objects/1.1 MiB.                                      
Copying file://tempFile.wav [Content-Type=audio/x-wav]...
/ [1/1 files][  5.4 MiB/  5.4 MiB] 100% Done                                    
Operation completed over 1 objects/5.4 MiB.                                      
Copying file://tempFileDegraded.wav [Content-Type=audio/x-wav]...
/ [1/1 files][  1.1 MiB/  1.1 MiB] 100% Done                                    
Operation completed over 1 objects/1.1 MiB.                                      
Copying file://tempFile.wav [Content-Type=audio/x-wav]...
/ [1/1 files][  5.4 MiB/  5.4 MiB] 100% Done                                    
Operation completed over 1 objects/5.4 MiB.                                      
Copying file://tempFileDegraded.wav [Content-Type=audio/x-wav]...
/ [1/1 files][  1.1 MiB/  1.1 MiB]

In [12]:
with open('eGMDdict3.pkl','wb') as f:
    pkl.dump(dictEGMD, f, pkl.HIGHEST_PROTOCOL)
    
with open('eGMDlabels3.pkl','wb') as f:
    pkl.dump(labels, f, pkl.HIGHEST_PROTOCOL)
    
# use gsutil to write both wav files to GCP bucket
!gsutil -m cp eGMDdict2.pkl gs://fyr-audio-data/raw-audio/reverseOrderPklFiles/eGMDdict3.pkl
!gsutil -m cp eGMDlabels2.pkl gs://fyr-audio-data/raw-audio/reverseOrderPklFiles/eGMDlabels3.pkl
    
#os.remove('eGMDdict.pkl')
#os.remove('eGMDlabels.pkl')


Copying file://eGMDdict2.pkl [Content-Type=application/octet-stream]...
/ [1/1 files][555.1 KiB/555.1 KiB] 100% Done                                    
Operation completed over 1 objects/555.1 KiB.                                    
Copying file://eGMDlabels2.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

/ [1/1 files][2